# Bidirectional LSTM example

[Original video](https://youtu.be/jGst43P-TJA)

Input of LSTM is Nx28x28 --> 28 time sequences with 28 features each.

N x time_sequence x features

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader

In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Hyperparameters
input_size = 28  # number of features for each time step
sequence_length = 28
num_layers = 2
hidden_size = 256  # number of nodes in each time step
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 2

In [ ]:
# Create a bidirectional LSTM
class BLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        # in MNIST batch goes first, thus batch_first=True
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,
                            batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)  # get only the last time sequence

    def forward(self, x):
        # init hidden state
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        # init serapate cell state
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        
        # forward propagation
        out, _ = self.lstm(x, (h0, c0))  # "_" => ignore hidden and cell states
        out = self.fc(out[:,-1,:])
        return out

In [ ]:
# Check if it runs correctly
# MNIST 28x28=784, output classes = 10
model = BLSTM(input_size=input_size, hidden_size=hidden_size,
            num_layers=num_layers, num_classes=num_classes).to(device)

x = torch.randn(batch_size, 28, 28).to(device)

# Run model on the input and print the shape
print(model(x).shape)

torch.Size([64, 10])


In [ ]:
# For the error: HTTPError: HTTP Error 403: Forbidden
# StackOverflow: https://stackoverflow.com/a/66461122/7550928
from six.moves import urllib    
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

# Load data
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw


Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
# Initialize network
model = BLSTM(input_size=input_size, hidden_size=hidden_size,
           num_layers=num_layers, num_classes=num_classes).to(device)

In [ ]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=learning_rate)

In [ ]:
# Train network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to Cuda if possible
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)

        # Forward
        scores = model(data)  # shape 64x10
        loss = criterion(scores, targets)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        # Gradient descent or adam step
        optimizer.step()

In [ ]:
# Check accuracy on training and test to see how good our model
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on training data')
    else:
        print('Checking accuracy on test data')
    num_correct = 0
    num_samples = 0
    model.eval()  # set model into evaluation mode

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            scores = model(x)  # shape = 64x10
            _, predictions = scores.max(dim=1)  # get index of max value
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    acc = float(num_correct) / float(num_samples) * 100
    print(f'Got {num_correct} / {num_samples} with accuracy {acc:.2f}')

    model.train()  # set model into train mode
    #return acc

In [ ]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 58865 / 60000 with accuracy 98.11
Checking accuracy on test data
Got 9796 / 10000 with accuracy 97.96
